In [ ]:

%pip install websockets pyaudio python-dotenv asyncio
%pip install --upgrade websockets

In [3]:
import os
import asyncio
import websockets
import json
import base64
import pyaudio
import sys
import time

# --- 1. 設定區 ---
API_KEY = os.getenv("OPENAI_API_KEY")
URL = "wss://api.openai.com/v1/realtime?model=gpt-realtime"

FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 24000
CHUNK = 1024
SYSTEM_INSTRUCTIONS = (
    "You are a professional interpreter. "
    "Task: Translate the user's input audio directly into English. "
    "Rules: "
    "1. Translate whatever the user says immediately. Even if it is a fragment or a command, translate it. "
    "2. If the user says '然後填寫其他資料', translate it as 'Then fill in other information'. "
    "3. If the input is pure silence or noise, **generate no text output** (return an empty string). "
    "4. NEVER output the word 'Nothing' as a status message."
)

class StrictOrderTranslator:

    def __init__(self):
        self.p = pyaudio.PyAudio()
        self.input_stream = None
        self.loop = None

        self.buffered_translation = ""  
        self.transcript_received = False
        self.prefix_printed = False      

    def setup_audio(self):
        self.input_stream = self.p.open(
            format=FORMAT, channels=CHANNELS, rate=RATE,
            input=True, frames_per_buffer=CHUNK,
        )

    async def send_audio(self, ws):
        print("🎤 翻譯模式啟動 (含檢查點)...")
        print("-" * 50)
        try:
            while True:
                data = await self.loop.run_in_executor(
                    None, self.input_stream.read, CHUNK, False
                )
                base64_audio = base64.b64encode(data).decode("utf-8")
                event = {
                    "type": "input_audio_buffer.append",
                    "audio": base64_audio
                }
                await ws.send(json.dumps(event))
        except Exception as e:
            print(f"❌ 傳送錯誤: {e}")

    async def receive_text(self, ws):
        async for message in ws:
            try:
                data = json.loads(message)
                event_type = data.get("type")

                # --- 0. 新的回合開始 ---
                if event_type == "response.created":
                    self.buffered_translation = ""
                    self.transcript_received = False
                    self.prefix_printed = False

                # --- 1. 收到原文 (Transcript) ---
                elif event_type == "conversation.item.input_audio_transcription.completed":
                    transcript = data.get("transcript", "").strip()
                    if transcript:
                        print(f"\n👂 [中文原文]: {transcript}")
               
                        # 🟢 [檢查點 Checkpoint] 🟢

                        print(f"✅ [檢查點]: 確認原文 '{transcript[:5]}...' 已載入模型，準備翻譯...")
                    
                        self.transcript_received = True
               
                        # 檢查是否有積壓的翻譯 (如果翻譯比原文早到)
                        if self.buffered_translation:
                            sys.stdout.write(f"🅰️  [英譯]: {self.buffered_translation}\n")
                            sys.stdout.flush()
                            self.buffered_translation = ""
                            self.prefix_printed = True

                # --- 2. 收到翻譯 (Output Delta) ---
                elif event_type == "response.text.delta":
                    text_delta = data.get("delta", "")               
                    if self.transcript_received:
                        if not self.prefix_printed:
                            sys.stdout.write("🅰️  [英譯]: ")
                            self.prefix_printed = True                   
                        sys.stdout.write(text_delta)
                        sys.stdout.flush()
                    else:
                        # 翻譯跑太快了，先存起來，等到檢查點通過後再印
                        self.buffered_translation += text_delta

                # --- 3. 翻譯結束 ---
                elif event_type == "response.text.done":
                    if self.prefix_printed:
                        print("\n") 
                elif event_type == "error":
                    print(f"\n🔥 API 錯誤: {data['error']['message']}")
            except Exception as e:
                print(f"❌ 接收錯誤: {e}")

    async def run(self):
        self.loop = asyncio.get_running_loop()
        if not API_KEY:
            print("❌ 錯誤: 找不到 OPENAI_API_KEY")
            return
        self.setup_audio()
        headers = {
            "Authorization": f"Bearer {API_KEY}",
            "OpenAI-Beta": "realtime=v1"
        }
        try:
            async with websockets.connect(URL, additional_headers=headers) as ws:
                print("✅ 已連線 Server")
                session_update = {
                    "type": "session.update",
                    "session": {
                        "modalities": ["text"],
                        "instructions": SYSTEM_INSTRUCTIONS,
                        "input_audio_transcription": {
                            "model": "whisper-1"
                        },
                        "turn_detection": {
                            "type": "server_vad",
                            "threshold": 0.6,
                            "prefix_padding_ms": 300,
                            "silence_duration_ms": 500
                        }
                    }
                }
                await ws.send(json.dumps(session_update))
                await asyncio.gather(
                    self.send_audio(ws),
                    self.receive_text(ws)
                )       
        except Exception as e:
            print(f"❌ 連線錯誤: {e}")
        finally:
            self.close()

    def close(self):
        print("正在關閉資源...")
        if self.input_stream and self.input_stream.is_active():
            self.input_stream.stop_stream()
            self.input_stream.close()
        self.p.terminate()

if __name__ == "__main__":
    if sys.platform.startswith('win'):
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
    translator = StrictOrderTranslator()
    try:
        await translator.run()
    except KeyboardInterrupt:
        print("\n🛑 使用者停止")
    except Exception as e:
        print(f"執行錯誤: {e}")

✅ 已連線 Server
🎤 翻譯模式啟動 (含檢查點)...
--------------------------------------------------

👂 [中文原文]: 喔 好像有了
✅ [檢查點]: 確認原文 '喔 好像有...' 已載入模型，準備翻譯...
🅰️  [英譯]: Oh, it seems to have it now.

👂 [中文原文]: 馬特要緊張看
✅ [檢查點]: 確認原文 '馬特要緊張...' 已載入模型，準備翻譯...
🅰️  [英譯]: Let me try talking.

👂 [中文原文]: Simple.
✅ [檢查點]: 確認原文 'Simpl...' 已載入模型，準備翻譯...

👂 [中文原文]: 國泰金寇
✅ [檢查點]: 確認原文 '國泰金寇...' 已載入模型，準備翻譯...
🅰️  [英譯]: Guotai Jingkou.

👂 [中文原文]: Cool.
✅ [檢查點]: 確認原文 'Cool....' 已載入模型，準備翻譯...

👂 [中文原文]: MBC 뉴스 이덕영입니다.
✅ [檢查點]: 確認原文 'MBC 뉴...' 已載入模型，準備翻譯...
🅰️  [英譯]: Jingkou. Haha.

👂 [中文原文]: Cięta... eee...
✅ [檢查點]: 確認原文 'Cięta...' 已載入模型，準備翻譯...

👂 [中文原文]: 但我覺得他專有名詞可能還是 應該是 他沒有辦法
✅ [檢查點]: 確認原文 '但我覺得他...' 已載入模型，準備翻譯...

👂 [中文原文]: 有人會講台語喔
✅ [檢查點]: 確認原文 '有人會講台...' 已載入模型，準備翻譯...
🅰️  [英譯]: But I think those technical terms might still—probably it can’t just
 directly turn into


👂 [中文原文]: 我不太好 好 大家多多指教
✅ [檢查點]: 確認原文 '我不太好 ...' 已載入模型，準備翻譯...
🅰️  [英譯]: Guotai, okay, let’s all try it together.

👂 [中文原文]: 台语
✅ [檢查點]: 確認原文 '台语...' 已

CancelledError: 